In [ ]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

In [ ]:
application_train <- read.csv('../input/application_train_clean_SMOTEd.csv',1)
dim(application_train)

In [ ]:
str(application_train)

In [ ]:
names(application_train)

In [ ]:
# convert categorical attributes to factor type
data_cat <- application_train[,c(1:49)]
data_num <- application_train[,-c(1:49)]
data_cat <- data.frame(apply(data_cat, 2,function(x){as.factor(x)}))
data_num <- data.frame(apply(data_num, 2,function(x){as.numeric(x)}))
application_train <- cbind(data_cat,data_num)
str(application_train)

In [ ]:
application_train <- application_train[,-c(12)] # FLAG_MOBIL has got just 1 level

In [ ]:
application_train$TARGET<-ifelse(application_train$TARGET=="1","yes","no")
application_train$TARGET<-factor(application_train$TARGET)

In [ ]:
# class distribution
table(application_train$TARGET)

In [ ]:
# Split Data into Train and test - 70:30
library(caret)
set.seed(7)
train_test_split<-createDataPartition(application_train$TARGET, p=0.7, list = FALSE)
train_data<-application_train[train_test_split,]
test_data<-application_train[-train_test_split,]

In [ ]:
# train - class distribution
table(train_data$TARGET)

In [ ]:
# test - class distribution
table(test_data$TARGET)

In [ ]:
# Support Vector Machine using caret
#trctrl <- trainControl(method = "cv", number = 5, summaryFunction=twoClassSummary, classProbs=TRUE)
#set.seed(7)
 
#svm_Linear <- train(TARGET ~., data = train_data[,-c(1)], method = "svmLinear",
#                 trControl=trctrl,
#                 preProcess = c("center", "scale"),
#                 metric="ROC")
#plot(svm_Linear)
#svm_Linear

In [ ]:
library(e1071)
# create model
svm_Linear <- svm(TARGET~.,data=train_data[,-c(1)],kernel='linear',gamma=0.2,cost=100)

In [ ]:
# Predict on train data 
pred_model_train <-predict(svm_Linear,newdata=train_data[,-c(1,2)])
confusionMatrix(pred_model_train, train_data$TARGET )

In [ ]:
# Prediction and Calculate Performance Metrics
predictions <- predict(svm_Linear,train_data[,-c(1,2)])
roc_curve <- roc(train_data$TARGET,as.numeric(predictions),plot=TRUE)
auc(roc_curve)

In [ ]:
# Predict on test data 
pred_model_test <-predict(svm_Linear,newdata=test_data[,-c(1,2)])
confusionMatrix(pred_model_test, test_data$TARGET )

In [ ]:
# Prediction and Calculate Performance Metrics
predictions <- predict(svm_Linear,test_data[,-c(1,2)])
roc_curve <- roc(test_data$TARGET,as.numeric(predictions),plot=TRUE)
auc(roc_curve)